In [ ]:
import sys
import subprocess

#import pkg_resources
#required = {'harmonypy','sklearn','scanpy','pandas', 'numpy', 'bbknn', 'scipy', 'matplotlib', 'seaborn' ,'scipy'}
#installed = {pkg.key for pkg in pkg_resources.working_set}
#missing = required - installed
#if missing:
#    print("Installing missing packages:" )
#    print(missing)
#    python = sys.executable
#    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

%matplotlib inline
from collections import Counter
from collections import defaultdict
import scanpy as sc
import pandas as pd
import pickle as pkl
import numpy as np
from bbknn import bbknn
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys
from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns; sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
import harmonypy as hm
from pathlib import Path

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=300, color_map='viridis')
plt.rcParams['figure.figsize'] = [10, 10]

# Plot General HSCPC Markers (FIG2A)

In [ ]:
adata = sc.read('./HSC_HSPC_combined_density_estimated.h5ad')

In [ ]:
adata_ipsc = sc.read('/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig3_Macrophages/Process_IPSC_mac_data/IPSC_DC_lin_use_this/v6_IPSC_mac_lin.h5ad')
adata_ipsc = adata_ipsc[adata_ipsc.obs['LR_assignment_v2'].str.contains('HSPC')]
sc.pp.scale(adata_ipsc,zero_center=True,max_value=10)

In [ ]:
adata_ipsc.obs['tissue'] = 'IPSC'
adata_ipsc.obs['dataset_annot'] = adata_ipsc.obs['LR_assignment_v2'].copy()
adata = adata.concatenate(adata_ipsc,join='inner')

In [ ]:
adata.obs.groupby(['dataset','fetal_ids']).apply(len)

In [ ]:
adata.obs.groupby(['dataset']).apply(len)

In [ ]:
'dataset_annot'

In [ ]:
general_hsc = {
'HSC':["CD34",	"MLLT3",	"SPINK2",	"HOPX",	"HLF",	"CLEC9A",'PROM1','HLA-DRA','HLA-DRB1',	'SOX17',"HOXA3",	"HOXA4",	"HOXA5",	"HOXA7",	"HOXA10","ICAM3",	"SELL",	"PLAC8",	"CD74"],#
}

general_hsc_surface_markers = {
'HSC_markers':["CD34",	"MLLT3",	"SPINK2",	"HOPX","MLLT3","MECOM",	"HLF","HLF"],#
'HSC_maturity_markers':["CLEC9A","FLT3","CD48",'PROM1','HLA-DRA','HLA-DRB1',],
'HSC_surface_tetraspaninis':['CD9','CD81','CD63','CD24','CD74'],
'HSC_maintenance':["SOX17","HOXA3",	"HOXA4",	"HOXA5",	"HOXA7",	"HOXA9","HOXA10"],
}


# HE_origin = {
# 'HSC':["KDR",	"FLT1",	"CDH5",],
# }

HE_origin_alternate_adhesion_combined = {
#'HSC':["PLVAP",	"PECAM1","ESAM",	"CDH5",	"FLT1","KDR",'ALDH1A1' , 'KCNK17',"ICAM3",	"SELL",	"PLAC8",	"CD74"],
#HSC':["PLVAP","ESAM","PECAM1",	"CDH5",	"FLT1","KDR", 'KCNK17',"ICAM3",	"SELL",	"PLAC8",	"CD74"],
'HSC':["PLVAP","ESAM","PECAM1",	"CDH5",	"FLT1","KDR", 'KCNK17'],
}

# YS_HSP_DE_difinitive = {
# 'HSC':["ICAM3",	"SELL",	"PLAC8",	"CD74",	"SOX17",],
# }

# YS_HSP_DE_difinitive_alternate = {
# 'HSC':["ICAM3",	"SELL",	"PLAC8",	"CD74",	"SOX17","CD63",	"CD37",],
# }


general_hsc_surface_markers_supp_HSC_survival = {
'HSC_surface_tetraspaninis':['CD9','CD81','CD63','CD24','CD74'],
'HSC_maturity_markers':['PROM1','HLA-DRA','HLA-DRB1',"CLEC9A","FLT3","CD48"],
}

In [ ]:
adata.obs['dataset_annot'].unique()

In [ ]:
# order these categories by lineage time
ordering =  ['Gastrula_HSPC_lineage_Yolk Sac_gastrulation',
 'YS_HSPC_1',
 'YS_HSPC_2',
 'AGM_10_HSPC',
 'EL_HSC',
 'FBM_HSC/MPP',
 'HSPC_1_IPSC',
'HSPC_2_IPSC'
]
adata.obs['dataset_annot'] = pd.Categorical(adata.obs['dataset_annot'], categories=ordering) 

In [ ]:
data_temp = adata
markers = HE_origin_alternate_adhesion_combined
spot_var = 'dataset_annot'
save_title = 'HE_origin_alternate_adhesion_combined_v5_with_IPSC'

(sc.pl.dotplot(data_temp, var_names = markers, groupby=spot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',save =save_title))
sc.pl.stacked_violin(data_temp ,var_names= markers, groupby=spot_var,  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale='var', var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, save=save_title, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None)

In [ ]:
data_temp = adata
markers = general_hsc
spot_var = 'dataset_annot'
save_title = 'general_hsc_v5_with_IPSC'

(sc.pl.dotplot(data_temp, var_names = markers, groupby=spot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',save =save_title))
sc.pl.stacked_violin(data_temp ,var_names= markers, groupby=spot_var,  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale='var', var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, save=save_title, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None)

In [ ]:
data_temp = adata
markers = HE_origin_alternate_adhesion_combined
spot_var = 'dataset_annot'
save_title = 'HE_origin_alternate_v5_with_IPSC'

(sc.pl.dotplot(data_temp, var_names = markers, groupby=spot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',save =save_title))
sc.pl.stacked_violin(data_temp ,var_names= markers, groupby=spot_var,  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale='var', var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, save=save_title, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None)

In [ ]:
adata = sc.read('/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_090321_notebooks/HSPC_comp/batch_by_organ_SCVI_HSPC_integration_output_batch_dataset_20220119.h5ad')

In [ ]:
gene_sets = {'metabolic' :['/nfs/team205/ig7/resources/GO-0045333_cell_respiration.csv'],
'cell_cycle' : ['/nfs/team205/ig7/resources/GO-0022402_cell_cycle_genes.csv'],
'cell_cell_adhesion':['/nfs/team205/ig7/resources/GO_0016337_cell_cell_adhesion.csv'],
'cell_matrix_adhesion': ['/nfs/team205/ig7/resources/GO_0007160_cell_matrix_adhesion.csv'],
'endo_cell_matrix_adhesion' :[ '/nfs/team205/ig7/resources/GO_0090673_endo_cell_matrix_adhesion.csv'],
'GNF2_SELL':['/nfs/team205/ig7/resources/GNF2_SELL.csv'],
'GNF2_ICAM3':['/nfs/team205/ig7/resources/GNF2_ICAM3.csv'],}
for i in gene_sets.keys():
    gene_list_loc = ''.join(str(e) for e in gene_sets[i])
    gene_list_name = i
    gene_list = list(pd.read_csv(gene_list_loc,header=None)[0])
    gene_list = list(adata.var.index[adata.var.index.isin(gene_list)])
    print(len(gene_list))
    sc.tl.score_genes(adata, gene_list, ctrl_size=50, gene_pool=None, n_bins=25, score_name=(gene_list_name + '_score'), random_state=0, copy=False, use_raw=None)
    
scores = adata.obs[[s + '_score' for s in list(gene_sets.keys())]]
scores['group'] = adata.obs['dataset']
scores.groupby('group').median()
sns.heatmap(scores.groupby('group').median(),  annot=True,vmin=-1, vmax=1, linewidths=1, center=0, square=True, cbar_kws={"shrink": 0.5})

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=10)
sc.tl.umap(adata)
sc.pl.umap(adata,color = 'dataset_annot')

In [ ]:
sc.pl.umap(adata,color = 'dataset_annot')

In [ ]:
sc.tl.embedding_density(adata, groupby='dataset_annot')
map_scores = []
for i in adata.obs['dataset_annot'].unique():
    # make bg blue
    adata.obs['umap_density_dataset_annot_'+i] = adata.obs['umap_density_dataset_annot']
    adata.obs.loc[~adata.obs['dataset_annot'].isin([i]),('umap_density_dataset_annot_'+i)] = 0.1
    #sc.pl.umap(adata,color = ('umap_density_dataset_annot_'+i),color_map = 'turbo')
    map_scores.append('umap_density_dataset_annot_'+i)
#sc.pl.umap(adata,color = 'umap_density_dataset_annot',color_map = 'turbo')
#sc.pl.embedding_density(adata, groupby='dataset_annot',color_map = 'seismic')
sc.pl.umap(adata,color = map_scores,color_map = 'turbo',legend_loc=None,save = ('all_density_HSC'))

In [ ]:
sc.tl.embedding_density(adata, groupby='dataset_annot')
map_scores = []
for i in adata.obs['dataset_annot'].unique():
    # make bg blue
    adata.obs['umap_density_dataset_annot_'+i] = adata.obs['umap_density_dataset_annot']
    adata.obs.loc[~adata.obs['dataset_annot'].isin([i]),('umap_density_dataset_annot_'+i)] = 0.1
    #sc.pl.umap(adata,color = ('umap_density_dataset_annot_'+i),color_map = 'turbo')
    map_scores.append('umap_density_dataset_annot_'+i)
    
    sc.pl.umap(adata,color = map_scores,color_map = 'turbo',legend_loc=None,save = (i+'_density_HSC'))
    
#sc.pl.umap(adata,color = 'umap_density_dataset_annot',color_map = 'turbo')
#sc.pl.embedding_density(adata, groupby='dataset_annot',color_map = 'seismic')
sc.pl.umap(adata,color = map_scores,color_map = 'turbo',legend_loc=None)

# cell cycle scoring

In [ ]:
gene_sets = {#'metabolic' :['/nfs/team205/ig7/resources/GO-0045333_cell_respiration.csv'],
'cell_cycle' : ['/nfs/team205/ig7/resources/GO-0022402_cell_cycle_genes.csv'],
# 'cell_cell_adhesion':['/nfs/team205/ig7/resources/GO_0016337_cell_cell_adhesion.csv'],
# 'cell_matrix_adhesion': ['/nfs/team205/ig7/resources/GO_0007160_cell_matrix_adhesion.csv'],
# 'endo_cell_matrix_adhesion' :[ '/nfs/team205/ig7/resources/GO_0090673_endo_cell_matrix_adhesion.csv'],
# 'GNF2_SELL':['/nfs/team205/ig7/resources/GNF2_SELL.csv'],
# 'GNF2_ICAM3':['/nfs/team205/ig7/resources/GNF2_ICAM3.csv'],}
            }
for i in gene_sets.keys():
    gene_list_loc = ''.join(str(e) for e in gene_sets[i])
    gene_list_name = i
    gene_list = list(pd.read_csv(gene_list_loc,header=None)[0])
    gene_list = list(adata.var.index[adata.var.index.isin(gene_list)])
    print(len(gene_list))
    sc.tl.score_genes(adata, gene_list, ctrl_size=50, gene_pool=None, n_bins=25, score_name=(gene_list_name + '_score'), random_state=0, copy=False, use_raw=None)

In [ ]:
cycle_2 = ['CDK1','MKI67']
sc.tl.score_genes(adata,cycle_2 , ctrl_size=50, gene_pool=None, n_bins=25, score_name=('cycle_2' + '_score'), random_state=0, copy=False, use_raw=None)
gene_sets ={
    'cell_cycle' : ['/nfs/team205/ig7/resources/GO-0022402_cell_cycle_genes.csv'],
    'cycle_2':[]
}

In [ ]:
# order these categories by lineage time
ordering =  ['Gastrula_HSPC_lineage_Yolk Sac_gastrulation',
 'YS_HSPC_1',
 'YS_HSPC_2',
 'AGM_10_HSPC',
 'EL_HSC',
 'FBM_HSC/MPP']
adata.obs['dataset_annot'] = pd.Categorical(adata.obs['dataset_annot'], categories=ordering) 

In [ ]:
scores = adata.obs[[s + '_score' for s in list(gene_sets.keys())]]
scores['group'] = adata.obs['dataset_annot']
scores_median = scores.groupby('group').median()
scores.groupby('group').median()

In [ ]:
sns_plot = sns.heatmap(scores_temp.groupby('group').median(),  annot=False,vmin=float(np.min(scores_temp.groupby('group').median())), vmax=float(np.max(scores_temp.groupby('group').median())), linewidths=1, center=float(np.mean(scores_temp.groupby('group').median())), square=True, cbar_kws={"shrink": 0.5},cmap = "vlag")
sns_plot.figure.savefig("./HPSC_prolif.pdf")

In [ ]:
concat = pd.concat([n,p,l],axis=1)
#remove mito genes from data
concat = concat[~concat["gene"].str.startswith("MT-")]
concat = concat[~concat["gene"].str.startswith("RP11-")]
concat["cluster"] = concat["cluster"].astype(str)
marker_df = concat.groupby('cluster').apply(lambda grp: grp.nlargest(20, 'logfc')).reset_index(drop=True)
marker_df = marker_df.groupby('cluster').apply(lambda grp: grp.nsmallest(10, 'pvals_adj')).reset_index(drop=True)
markers = marker_df.groupby('cluster')['gene'].apply(list).to_dict()
markers = dict(sorted(markers.items()))
markers = {str(k):v for k,v in markers.items()}
data_temp = YS_HSC 
markers = markers
(sc.pl.dotplot(data_temp, var_names = markers, groupby=spot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',save= 'clotting_over_time'))

# Dotplots for HSC_surival _cut

In [ ]:
YS_HSC = adata[adata.obs['dataset_annot'].isin(['Gastrula_HSPC_lineage_Yolk Sac_gastrulation', 'YS_HSPC_1', 'YS_HSPC_2'])]

In [ ]:
YS_HSC.obs['stage_groups'] = YS_HSC.obs['dataset_annot'][:]
YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage_groups'].astype(str)
stages= ['CS10', 'CS11', 'CS22', 'CS23', 'CS14', 'CS18', 'CS15']
groups = ['CS10_11', 'CS10_11', 'CS22_23', 'CS22_23', 'CS14_15', 'CS18', 'CS14_15']
dic = dict(zip(stages,groups))
#YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage'].map(dic)
YS_HSC.obs.loc[YS_HSC.obs['stage'].isin(stages),'stage_groups'] = YS_HSC.obs['stage'].map(dic)
YS_HSC.obs['stage_groups'].unique()
# order these categories by lineage time
YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage_groups'].str.replace('Gastrula_HSPC_lineage_Yolk Sac_gastrulation','CS7')

ordering =  ['CS7', 'CS10_11', 'CS14_15', 'CS18','CS22_23']
YS_HSC.obs['stage_groups'] = pd.Categorical(YS_HSC.obs['stage_groups'], categories=ordering) 

In [ ]:
lineage_bias = {
'YS_HSPC_migration':['CD9','CD81','IGFBP2','RAC1','RAC2','ADAM10','RUNX1'],
'EMP_generation':['EZH2'],
    'HSC_maturity_markers':["CLEC9A","FLT3","CD48",'PROM1','HLA-DRA','HLA-DRB1',],
}

In [ ]:

lineage_bias = {
'YS_HSPC_migration':['CD9','CD81','IGFBP2','RAC1','RAC2','RUNX1'],
'EMP_generation':['EZH2'],
'Migration':['ADAM10'],
# 'HSC_maturity_markers':["CD48",'PROM1','HLA-DRA',],
}
data = YS_HSC[:]
plot_name = 'survival_over_time'
plot_var = 'stage_groups'
markers = lineage_bias

sc.pl.matrixplot(data, var_names = markers, groupby=plot_var, dendrogram=False,
                 colorbar_title='mean z-score',  vmin=-2, vmax=3, vcenter=-0.2,cmap='RdBu_r',swap_axes=False,save = plot_name)
(sc.pl.dotplot(data, var_names = markers, groupby=plot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',swap_axes=True,save = plot_name,dot_max=0.5,figsize =[4,5]))
sc.pl.stacked_violin(data ,var_names= markers, groupby=plot_var, log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale=None, var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=True, show=None, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None,save = plot_name)
#YS_HSC, var_names = lineage_bias, groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var')
sc.pl.tracksplot(data ,var_names= markers, groupby=plot_var, use_raw=None, log=False, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, layer=None, show=None, save=None )

# Dotplots for HSC_surival full

In [ ]:
lineage_bias = {
'YS_HSPC_migration':['CD9','CD81','IGFBP2','RAC1','RAC2'],
'EMP_generation':['EZH2'],
'transmigration':['ADAM10'],
'Mega_k_transition':['ITGA2B','FLI1','TPT1'],
'ery_transition_inhibition':['CD24','RUNX3','SERPINB1'],
'HB_genes':['HBZ','HBA1','HBG1','HBE1'],
'ery_bias':['GATA1','KLF1','ALAS2'],
'survival':['IGFBP2','THPO'],
'CD74':['CD74'],
'long_term_engraftment':['CD63','SOX17'],
#'erypsis':["TAL1",	"LMO2",	"LDB1",	"GATA1",	"MYB",	"ZFPM1",	"KLF1",	"KLF3",	"KLF11",	"FLI1",	"RB1",	"SPI1",	"EPOR",	"STAT5A",	"TRIM10",	"GATA2",],
#'mkpsis':["RUNX1",	"MPL",	"NFE2",],
}

lineage_bias = {
'YS_HSPC_migration':['CD9','CD81','IGFBP2','RAC1','RAC2'],
'EMP_generation':['EZH2'],
'transmigration':['ADAM10','PLAC8','ICAM1','SELL'],
#'erypsis':["TAL1",	"LMO2",	"LDB1",	"GATA1",	"MYB",	"ZFPM1",	"KLF1",	"KLF3",	"KLF11",	"FLI1",	"RB1",	"SPI1",	"EPOR",	"STAT5A",	"TRIM10",	"GATA2",],
#'mkpsis':["RUNX1",	"MPL",	"NFE2",],
}

In [ ]:
YS_HSC = adata[adata.obs['dataset_annot'].isin(['Gastrula_HSPC_lineage_Yolk Sac_gastrulation', 'YS_HSPC_1', 'YS_HSPC_2'])]

In [ ]:
YS_HSC.obs['stage_groups'] = YS_HSC.obs['dataset_annot'][:]
YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage_groups'].astype(str)
stages= ['CS10', 'CS11', 'CS22', 'CS23', 'CS14', 'CS18', 'CS15']
groups = ['CS10_11', 'CS10_11', 'CS22_23', 'CS22_23', 'CS14_15', 'CS18', 'CS14_15']
dic = dict(zip(stages,groups))
#YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage'].map(dic)
YS_HSC.obs.loc[YS_HSC.obs['stage'].isin(stages),'stage_groups'] = YS_HSC.obs['stage'].map(dic)
YS_HSC.obs['stage_groups'].unique()
# order these categories by lineage time
YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage_groups'].str.replace('Gastrula_HSPC_lineage_Yolk Sac_gastrulation','CS7')


ordering =  ['CS7', 'CS10_11', 'CS14_15', 'CS18','CS22_23']
YS_HSC.obs['stage_groups'] = pd.Categorical(YS_HSC.obs['stage_groups'], categories=ordering) 

In [ ]:
#YS_HSC.obs.groupby(['stage_groups','component',]).count()

In [ ]:
lineage_bias = {
'YS_HSPC_migration':['CD9','CD81','IGFBP2','RAC1','RAC2','ADAM10','RUNX1'],
'EMP_generation':['EZH2'],
'transmigration':['PLAC8','ICAM1','SELL','CCL4','CXCR4','CXCL12'],
#'LT_HSC_BM_homing':['CCR3','CCR9',,'IL18','KIT','CDH5'],
#'erypsis':["TAL1",	"LMO2",	"LDB1",	"GATA1",	"MYB",	"ZFPM1",	"KLF1",	"KLF3",	"KLF11",	"FLI1",	"RB1",	"SPI1",	"EPOR",	"STAT5A",	"TRIM10",	"GATA2",],
#'mkpsis':["RUNX1",	"MPL",	"NFE2",],
}

In [ ]:
sc.pl.matrixplot(YS_HSC, var_names = lineage_bias, groupby='stage_groups', dendrogram=False,
                 colorbar_title='mean z-score',  vmin=-2, vmax=3, vcenter=-0.2,cmap='RdBu_r',swap_axes=False,save = 'HSC_surival_migration_adhesion')

(sc.pl.dotplot(YS_HSC, var_names = lineage_bias, groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var',swap_axes=True,save = 'HSC_surival_migration_adhesion'))
sc.pl.stacked_violin(YS_HSC ,var_names= lineage_bias, groupby='stage_groups',  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale=None, var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None,save = 'HSC_surival_migration_adhesion')
#YS_HSC, var_names = lineage_bias, groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var')
sc.pl.tracksplot(YS_HSC ,var_names= lineage_bias, groupby='stage_groups', use_raw=None, log=False, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, layer=None, show=None, save=None )

# Dotplots for Liver & AGM & BM too

In [ ]:
adata.obs['dataset'].unique()

In [ ]:
adata.obs['dataset'].unique()

In [ ]:
adata.obs['stage_groups'] = adata.obs['dataset_annot'][:]
adata.obs['stage_groups'] = adata.obs['stage_groups'].astype(str)
stages= ['CS10', 'CS11', 'CS22', 'CS23', 'CS14', 'CS18', 'CS15']
groups = ['CS10_11', 'CS10_11', 'CS22_23', 'CS22_23', 'CS14_15', 'CS18', 'CS14_15']
dic = dict(zip(stages,groups))
#adata.obs['stage_groups'] = adata.obs['stage'].map(dic)
adata.obs.loc[adata.obs['stage'].isin(stages),'stage_groups'] = adata.obs['stage'].map(dic)
adata.obs['stage_groups'].unique()

# order these categories by lineage time
ordering =  ['Gastrula_HSPC_lineage_Yolk Sac_gastrulation', 'CS10_11', 'CS14_15', 'CS18','CS22_23', 'EL_HSC', 'FBM_HSC/MPP']
adata.obs['stage_groups'] = pd.Categorical(adata.obs['stage_groups'], categories=ordering) 

In [ ]:
adata_no_hsc = adata[~adata.obs['dataset'].isin(['YS','Gastrula'])]

In [ ]:
data = adata_no_hsc[:]
plot_name = 'combined_surival_migration_adhesion'
plot_var = 'dataset'
markers = lineage_bias

sc.pl.matrixplot(data, var_names = markers, groupby=plot_var, dendrogram=False,
                 colorbar_title='mean z-score',  vmin=-2, vmax=3, vcenter=-0.2,cmap='RdBu_r',swap_axes=False,save = plot_name)
(sc.pl.dotplot(data, var_names = markers, groupby=plot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',swap_axes=True,save = plot_name,dot_max=0.5,figsize =[5,5]))
sc.pl.stacked_violin(data ,var_names= markers, groupby=plot_var,  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale=None, var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None,save = plot_name)
#YS_HSC, var_names = lineage_bias, groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var')
sc.pl.tracksplot(data ,var_names= markers, groupby=plot_var, use_raw=None, log=False, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, layer=None, show=None, save=None )

In [ ]:
lineage_bias = {
'YS_HSPC_migration':['CD9','CD81','IGFBP2','RAC1','RAC2','ADAM10','RUNX1'],
'EMP_generation':['EZH2'],
}

In [ ]:
data = adata_no_hsc[:]
plot_name = 'cut_version_combined_surival_migration_adhesion'
plot_var = 'dataset'
markers = lineage_bias

sc.pl.matrixplot(data, var_names = markers, groupby=plot_var, dendrogram=False,
                 colorbar_title='mean z-score',  vmin=-2, vmax=3, vcenter=-0.2,cmap='RdBu_r',swap_axes=False,save = plot_name)
(sc.pl.dotplot(data, var_names = markers, groupby=plot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',swap_axes=True,save = plot_name,dot_max=0.5,figsize =[5,5]))
sc.pl.stacked_violin(data ,var_names= markers, groupby=plot_var,  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale=None, var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None,save = plot_name)
#YS_HSC, var_names = lineage_bias, groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var')
sc.pl.tracksplot(data ,var_names= markers, groupby=plot_var, use_raw=None, log=False, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, layer=None, show=None, save=None )

In [ ]:
EL_stages = {
'F61':'CS18',
'F35':'CS22_23', #CS23
'F32':'CS22_23', #CS22
}
EL_adata = adata[adata.obs['dataset'].isin(['EL'])]
EL_adata.obs['stage'] = EL_adata.obs['fetal_ids'].map(EL_stages)

ordering =  ['CS7','CS10_11','CS18','CS22_23']
EL_adata.obs['stage'] = pd.Categorical(EL_adata.obs['stage'], categories=ordering) 

In [ ]:
EL_adata.obs['stage'].unique()

In [ ]:
data = EL_adata[:]
plot_name = 'fliv_surival_migration_adhesion'
plot_var = 'stage'
markers = lineage_bias

sc.pl.matrixplot(data, var_names = markers, groupby=plot_var, dendrogram=False,
                 colorbar_title='mean z-score',  vmin=-2, vmax=3, vcenter=-0.2,cmap='RdBu_r',swap_axes=False,save = plot_name)
(sc.pl.dotplot(data, var_names = markers, groupby=plot_var,dendrogram=False, color_map='Reds', show = True,standard_scale='var',swap_axes=True,save = plot_name,dot_max=0.5,figsize =[3,5]))
sc.pl.stacked_violin(data ,var_names= markers, groupby=plot_var,  log=False, use_raw=None, num_categories=7, title=None, colorbar_title='Median exp', figsize=None, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, standard_scale=None, var_group_rotation=None, layer=None, stripplot=False, jitter=False, size=1, scale='width', yticklabels=False, order=None, swap_axes=False, show=None, return_fig=False, row_palette=None, cmap='Reds', ax=None, vmin=None, vmax=None, vcenter=None, norm=None,save = plot_name)
#YS_HSC, var_names = lineage_bias, groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var')
sc.pl.tracksplot(data ,var_names= markers, groupby=plot_var, use_raw=None, log=False, dendrogram=False, gene_symbols=None, var_group_positions=None, var_group_labels=None, layer=None, show=None, save=None )

### Immature erythroblasts with extensive ex vivo self-renewal capacity emerge from the early mammalian fetus
- mayb 2 waves of erythroid production?? propr to gastrula data and after gastrula data - self renewing
- Sdf-1 (CXCL12) induces CD9 expression 

# try to order by stage

In [ ]:
scanpy.tl.diffmap(adata, n_comps=15, neighbors_key=None, random_state=0, copy=False)

In [ ]:
adata.obs['stage_groups'] = YS_HSC.obs['dataset_annot'][:]
YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage_groups'].astype(str)
stages= ['CS10', 'CS11', 'CS22', 'CS23', 'CS14', 'CS18', 'CS15']
groups = ['CS10_11', 'CS10_11', 'CS22_23', 'CS22_23', 'CS14_15', 'CS18', 'CS14_15']
dic = dict(zip(stages,groups))
#YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage'].map(dic)
YS_HSC.obs.loc[YS_HSC.obs['stage'].isin(stages),'stage_groups'] = YS_HSC.obs['stage'].map(dic)

In [ ]:
sc.tl.embedding_density(adata, groupby='dataset_annot')
map_scores = []
for i in adata.obs['dataset_annot'].unique():
    # make bg blue
    adata.obs['umap_density_dataset_annot_'+i] = adata.obs['umap_density_dataset_annot']
    adata.obs.loc[~adata.obs['dataset_annot'].isin([i]),('umap_density_dataset_annot_'+i)] = 0.1
    #sc.pl.umap(adata,color = ('umap_density_dataset_annot_'+i),color_map = 'turbo')
    map_scores.append('umap_density_dataset_annot_'+i)
#sc.pl.umap(adata,color = 'umap_density_dataset_annot',color_map = 'turbo')
#sc.pl.embedding_density(adata, groupby='dataset_annot',color_map = 'seismic')
sc.pl.umap(adata,color = map_scores,color_map = 'turbo',legend_loc=None)

In [ ]:
adata.obs['dataset_annot'].unique()

# DE in HSPC for GSEA over time

In [ ]:
gene_sets = {'metabolic' :['/nfs/team205/ig7/resources/GO-0045333_cell_respiration.csv'],
'cell_cycle' : ['/nfs/team205/ig7/resources/GO-0022402_cell_cycle_genes.csv'],
'cell_cell_adhesion':['/nfs/team205/ig7/resources/GO_0016337_cell_cell_adhesion.csv'],
'cell_matrix_adhesion': ['/nfs/team205/ig7/resources/GO_0007160_cell_matrix_adhesion.csv'],
'endo_cell_matrix_adhesion' :[ '/nfs/team205/ig7/resources/GO_0090673_endo_cell_matrix_adhesion.csv'],
'GNF2_SELL':['/nfs/team205/ig7/resources/GNF2_SELL.csv'],
'GNF2_ICAM3':['/nfs/team205/ig7/resources/GNF2_ICAM3.csv'],}
for i in gene_sets.keys():
    gene_list_loc = ''.join(str(e) for e in gene_sets[i])
    gene_list_name = i
    gene_list = list(pd.read_csv(gene_list_loc,header=None)[0])
    gene_list = list(adata.var.index[adata.var.index.isin(gene_list)])
    print(len(gene_list))
    sc.tl.score_genes(YS_HSC, gene_list, ctrl_size=50, gene_pool=None, n_bins=25, score_name=(gene_list_name + '_score'), random_state=0, copy=False, use_raw=None)
    
scores = YS_HSC.obs[[s + '_score' for s in list(gene_sets.keys())]]
scores['group'] = YS_HSC.obs['stage_groups']
scores.groupby('group').median()

In [ ]:
gene_list_name + '_score'

## Plot HB genes across time starting from early gastrula ery

In [ ]:
YS_HSC.obs['celltype_for_comparison'].unique()

In [ ]:
scores = YS_HSC.obs[[s + '_score' for s in list(gene_sets.keys())]]
scores['group'] = YS_HSC.obs['stage_groups']
scores.groupby('group').median()

In [ ]:
sns.heatmap(scores.groupby('group').median(),  annot=True,vmin=-1, vmax=1, linewidths=1, center=0, square=True, cbar_kws={"shrink": 0.5})

In [ ]:
sc.tl.rank_genes_groups(YS_HSC, 'stage_groups', method='wilcoxon',n_genes=500)
result = YS_HSC.uns['rank_genes_groups']
groups = result['names'].dtype.names
DE = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals','logfoldchanges']}).head(500)

DE_name ="./DEGS_YS_HSC_overtime.csv"
DE.to_csv(DE_name)

degs = DE[:]

In [ ]:
degs = DE[:]

if 'concat' in locals() or 'concat' in globals():
    del(concat)
n = degs.loc[:, degs.columns.str.endswith("_n")]
n = pd.melt(n)
p = degs.loc[:, degs.columns.str.endswith("_p")]
p = pd.melt(p)
l = degs.loc[:, degs.columns.str.endswith("_l")]
l = pd.melt(l)
n = n.replace(regex=r'_n', value='')

n = n.rename(columns={"variable": "cluster", "value": "gene"})
p = (p.drop(["variable"],axis = 1)).rename(columns={ "value": "p_val"})
l = (l.drop(["variable"],axis = 1)).rename(columns={ "value": "logfc"})

concat = pd.concat([n,p,l],axis=1)

#remove mito genes from data
concat = concat[~concat["gene"].str.startswith("MT-")]
concat = concat[~concat["gene"].str.startswith("RP11-")]
concat = concat[~concat["gene"].str.startswith("RPL")]
concat = concat[~concat["gene"].str.startswith("RPS")]

concat["cluster"] = concat["cluster"].astype(str)
marker_df = concat.groupby('cluster').apply(lambda grp: grp.nsmallest(20, 'p_val')).reset_index(drop=True)
marker_df = marker_df.groupby('cluster').apply(lambda grp: grp.nlargest(5, 'logfc')).reset_index(drop=True)
markers = marker_df.groupby('cluster')['gene'].apply(list).to_dict()
markers = dict(sorted(markers.items()))
markers = {str(k):v for k,v in markers.items()}

In [ ]:
markers

In [ ]:
markers ={'CS7': ['U2AF1L5', 'C11orf98', 'MRPL38', 'DDX39B', 'SARNP'],
'CS10_11': ['IGFBP2', 'SLC16A3', 'BNIP3', 'MAP2K2', 'PSMA2'],
 'CS14_15': ['GPX1', 'AC090498.1', 'MTRNR2L12', 'ATP6V0C', 'UQCRHL'],
 'CS18': ['MTRNR2L12', 'GPX1', 'AC090498.1', 'ATP6V0C', 'UQCRHL'],
 'CS22_23': ['ALB', 'AC104389.1', 'AREG', 'LUM', 'FRZB'],}

In [ ]:
(sc.pl.dotplot(YS_HSC, var_names = markers,groupby='stage_groups',dendrogram=False, color_map='Reds', show = True,standard_scale='var',swap_axes=False,save = 'HSC_DEGS_by_time'))

In [ ]:
concat

In [ ]:
import gseapy
gene_set_names = gseapy.get_library_name(database='Human')
glist = list(concat['gene'][concat['cluster'].isin(['CS7'])])
#glist = list(concat['gene'][concat['cluster'].isin(['monocyte_Yolk Sac_gastrulation'])])

In [ ]:
# order these categories by lineage time
ordering =  ['Gastrula_HSPC_lineage_Yolk Sac_gastrulation', 'CS10_11', 'CS14_15', 'CS18','CS22_23','AGM_10_HSPC', 'EL_HSC', 'FBM_HSC/MPP']
adata.obs['stage_groups'] = pd.Categorical(adata.obs['stage_groups'], categories=ordering) 

In [ ]:
adata.obs['stage_groups'].unique()

# Reference main ys obj

In [ ]:
ys_main = sc.read('/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_data/A4_V7_YS_integrated_data_singlets_with_raw_counts_for_MS_plotting_20211111_with_obsp.h5ad')

# Proportions of each population by age

In [ ]:
list(ys_main.obs['cell.labels'].unique())

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]

In [ ]:
keep = [ 'HSPC_1',
'HSPC_2',
'MEMP',
 'Early_Erythroid',
 'Erythroid',
 'MK',]
ys_main.obs['cell.labels'] = ys_main.obs['cell.labels'].astype(str)
ys_main.obs['stage_groups'] = ys_main.obs['cell.labels'][:]
ys_main.obs['stage_groups'] = ys_main.obs['stage_groups'].astype(str)
stages= ['CS10', 'CS11', 'CS22', 'CS23', 'CS14', 'CS18', 'CS15']
groups = ['CS10_11', 'CS10_11', 'CS22_23', 'CS22_23', 'CS14_15', 'CS18', 'CS14_15']
dic = dict(zip(stages,groups))
#YS_HSC.obs['stage_groups'] = YS_HSC.obs['stage'].map(dic)
ys_main.obs.loc[ys_main.obs['stage'].isin(stages),'stage_groups'] = ys_main.obs['stage'].map(dic)
ys_main.obs['stage_groups'].unique()

obs = ys_main.obs[ys_main.obs['cell.labels'].isin(keep)]
obs['cell.labels'] = pd.Categorical(obs['cell.labels'], categories=keep) 

count1 = pd.DataFrame((obs.groupby(['stage_groups','cell.labels'], observed=True).count()).iloc[:,0])
count1 = count1.reset_index() 
for i in count1['stage_groups']:
    total = np.sum(count1.loc[count1['stage_groups'].isin([i]),'stage'])
    count1.loc[count1['stage_groups'].isin([i]),'stage'] = (count1.loc[count1['stage_groups'].isin([i]),'stage']/total)*100
# Reorder categories
count1['cell.labels'] = pd.Categorical(count1['cell.labels'], categories=keep) 
# Reorder categories
count1.pivot_table('stage','stage_groups','cell.labels').plot.bar(stacked=True, figsize=(10,10)).legend(loc='upper right')

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]

In [ ]:
sc.pl.umap(adata,color = 'dataset_annot',palette = col_dic,save = 'HSPC_combined_color_umap')

In [ ]:
sc.tl.embedding_density(adata, groupby='dataset_annot')
map_scores = []
for i in adata.obs['dataset_annot'].unique():
    # make bg blue
    adata.obs['umap_density_dataset_annot_'+i] = adata.obs['umap_density_dataset_annot']
    adata.obs.loc[~adata.obs['dataset_annot'].isin([i]),('umap_density_dataset_annot_'+i)] = 0.1
    #sc.pl.umap(adata,color = ('umap_density_dataset_annot_'+i),color_map = 'turbo')
    map_scores.append('umap_density_dataset_annot_'+i)
#sc.pl.umap(adata,color = 'umap_density_dataset_annot',color_map = 'turbo')
#sc.pl.embedding_density(adata, groupby='dataset_annot',color_map = 'seismic')
sc.pl.umap(adata,color = map_scores,color_map = 'turbo',legend_loc=None)

In [ ]:
sc.tl.embedding_density(adata, groupby='dataset_annot')
map_scores = []
for i in adata.obs['dataset_annot'].unique():
    # make bg blue
    adata.obs['umap_density_dataset_annot_'+i] = adata.obs['umap_density_dataset_annot']
    adata.obs.loc[~adata.obs['dataset_annot'].isin([i]),('umap_density_dataset_annot_'+i)] = 0.1
    #sc.pl.umap(adata,color = ('umap_density_dataset_annot_'+i),color_map = 'turbo')
    map_scores.append('umap_density_dataset_annot_'+i)
#sc.pl.umap(adata,color = 'umap_density_dataset_annot',color_map = 'turbo')
#sc.pl.embedding_density(adata, groupby='dataset_annot',color_map = 'seismic')
sc.pl.umap(adata,color = map_scores,color_map = 'turbo',legend_loc=None)

In [ ]:
adata.write('./HSC_HSPC_combined_density_estimated.h5ad')

### Itga2b regulation at the onset of definitive hematopoiesis and commitment to differentiation
CD41 contributes to hematopoietic stem cell (HSC) and megakaryocyte/platelet functions. CD41 expression marks the onset of definitive hematopoiesis in the embryo where it participates in regulating the numbers of multipotential progenitors. Key to platelet aggregation, CD41 expression also characterises their precursor, the megakaryocyte, and is specifically up regulated during megakaryopoiesis. Though phenotypically unique, megakaryocytes and HSC share numerous features, including key transcription factors, which could indicate common sub-regulatory networ

### CD9
- Tetraspanin CD9 has also been described as a regulator of HSPC adhesion and homing. Pretreatment of cord blood CD34+ cells with an anti-CD9 monoclonal antibody inhibited CXCL12-mediated transendothelial migration; however, adhesion to fibronectin and endothelial cells was enhanced (Leung et al., 2011). 
- CD9 blockage inhibits homing to BM and spleen
- Additionally, antibody pretreatment of CD34+ cells significantly impaired their homing to the BM and sorted CD34+ CD9– cells displayed lower BM homing capacity compared with that of total CD34+ cells.
* **Tetraspanin Scaffold Proteins Function as Key Regulators of Hematopoietic Stem Cells https://www.frontiersin.org/articles/10.3389/fcell.2020.00598/full**
* **The tetraspanin CD9 regulates migration, adhesion, and homing of human cord blood CD34+ hematopoietic stem and progenitor cells**
* **Differential Expression of the Tetraspanin CD9 in Normal and Leukemic Stem Cells**
* **CD9, a key actor in the dissemination of lymphoblastic leukemia, modulating CXCR4-mediated migration via RAC1 signaling**
- After C-X-C motif chemokine ligand 12 (CXCL12) stimulation, CD9 promoted the formation of long cytoplasmic actin-rich protrusions. We demonstrated that CD9 enhanced RAC1 activation, in both REH cells and blasts from patients. 
- inhibition of RAC1 activation decreased migration in vitro,

* **Disruption of CD9 Expression Affects Adhesion, Migration, and Actin Polymerization through RAC1 Signalling Pathway in ETV6/RUNX1 Pre-B Lymphocytes**
- demonstrated that cells depleted in CD9 have lost their ability to activate RAC1 in response to CXCL12


### RAC1
- Disruption of CD9 Expression Affects Adhesion, Migration, and Actin Polymerization through RAC1 Signalling Pathway in ETV6/RUNX1 Pre-B Lymphocytes.
* **Rac1 Regulates Migration of Yolk Sac Hematopoietic Progenitors into the Blood, a Process Essential for Subsequent Seeding of Fetal Liver Hematopoiesis.**
- Rac1/2 and cdc42 deletion prevents FL HSC from reaching the marrow for transition of hematopoiesis, consistent with their importance in upstream regulation of actin polymerization, epistatically above both WAVE and WASP
* **The Wave2 scaffold Hem-1 is required for transition of fetal liver hematopoiesis to bone marrow**

- Rac1 is dispensable for steady-state hematopoiesis
- RAC1- demonstrated defective proliferative signaling via the receptor tyrosine kinase, c-kit, while loss of Rac2 activity leads to a pro-apoptotic phenotype
* **On how Rac controls hematopoietic stem cell activity**

- Rac2-deficient HSCs show defective migration and adhesion resulting in reduced retention in the marrow and increased mobilization into circulation.26  Rac1-deficient HSCs/Ps demonstrate decreased homing to BM and impaired endosteal microlocalization upon transplantation. However, if already resident in BM, Rac1 deficiency does not impair steady-state hematopoiesis, suggesting Rac1 is a key molecular switch regulating engraftment.21 
* **Rac1 is essential for intraembryonic hematopoiesis and for the initial seeding of fetal liver with definitive hematopoietic progenitor cells**

### CD81
- Pull-down experiments revealed that CD81-Rac interaction was direct and independent of Rac activation status. Knockdown of CD81 resulted in enhanced protrusion rate, altered focal adhesion formation, and decreased cell migration, correlating with increased active Rac.
* **CD81 regulates cell migration through its association with Rac GTPase**
* **CD81 is essential for the formation of membrane protrusions and regulates Rac1-activation in adhesion-dependent immune cell migration**

### CD74
- Here, we demonstrate that mice lacking CD74 exhibit an accumulation of HSCs in the bone marrow (BM) due to their increased potential to repopulate and compete for BM niches.
-    CD74 is a regulator of hematopoietic stem cell maintenance:: lack of CD74, leads to lack of HSC migration

### TPT1
- Megakaryocytic lineage transition
- overexpression remarkably induced cell proliferation, migration and invasion, and significantly attenuated cell adhesion ability in vitro in epithelial ovarian cancer through TPT1 and downstream PI3K/AKT signaling
-    LncRNA TPT1-AS1 promotes tumorigenesis and metastasis in epithelial ovarian cancer by inducing TPT1 expression.

### CD24
- Inhibits Erythroid differentiation TNF-induced CD24 expression at the RNA and protein level
- The Role of CD24 in TNF-Alpha-Inhibited Erythroid Differentiation and Its Signaling Pathway

### Cd63
- tetraspanin CD63 was found to interact with the tissue inhibitor of metalloproteinase-1 (TIMP1), a protein important for HSC quiescence and long-term engraftment (Jung et al., 2006). TIMP1 was shown to bind to the CD63/β1 integrin complex on the surface of human CD34+ HSPCs to induce adhesion and migration (Wilk et al., 2013). Moreover, this group determined that homing and short-term engraftment of HSPCs were also increased upon exogenous stimulation with TIMP1. The interaction of TIMP1 and CD63 was also shown to impact HSPC proliferation through the activation of the PI3K/AKT signaling pathway

- CD34+ HSPCs to induce adhesion and migration (Wilk et al., 2013). Moreover, this group determined that homing and short-term engraftment of HSPCs were also increased upon exogenous stimulation with TIMP1. The interaction of TIMP1 and CD63 was also shown to impact HSPC proliferation through the activation of the PI3K/AKT signaling pathway (Rossi et al., 2015) and the enrichment of myeloid progenitors, impacting granulopoiesis (Kobuch et al.,

### EZh2
- Ezh2 is essential for YS hematopoiesis but dispensable for subsequent aorta–gonad–mesonephros (AGM) blood development. Loss of EZH2 activity in hemogenic endothelium (HE) leads to the generation of phenotypically intact but functionally deficient erythro-myeloid progenitors (EMPs), while the generation of primitive erythroid cells is not affected.
* **Ezh2 is essential for the generation of functional yolk sac derived erythro-myeloid progenitors **

### IGFBP2
* **IGF binding protein 2 supports the survival and cycling of hematopoietic stem cells**

# Extrinsic modifiers

### NOTCH1
- Notch signaling was inhibited by CD9 knockdown or CD9 antibody in cell lines. Finally, CD9 antibody showed anti-tumor effects in cell proliferation MTT assay, transwell migration assay and colony formation assay.
* **Tetraspanin CD9 interacts with α-secretase to enhance its oncogenic function in pancreatic cancer**

* **Notch1 regulates progenitor cell proliferation and differentiation during mouse yolk sac hematopoiesis**
* **Notch Stimulates Both Self-Renewal and Lineage Plasticity in a Subset of Murine CD9High Committed Megakaryocytic Progenitors**